In [66]:
import pandas as pd
import os
import numpy as np
pd.options.display.max_columns=199

In [67]:
source_list_describe=pd.read_excel("/home/jian/Projects/Red Nose Day/source_code_list_updated.xlsx",sheetname="source_code",dtype=str)
origin_source_list_describe=pd.read_excel("/home/jian/Projects/Red Nose Day/source_code_list_updated.xlsx",sheetname="origin_source_code",dtype=str)
origin_source_list_describe.columns=['Origin Source Code','Description']

In [68]:
df_x=source_list_describe
df_x.columns=['ActivistCodeName','Description']
df_y=origin_source_list_describe
df_y.columns=['ActivistCodeName','Description']

source_list=pd.concat([df_x,df_y],ignore_index=True)
source_list.drop_duplicates(inplace=True)

source_list_describe.columns=['Source Code','Description']
origin_source_list_describe.columns=['Origin Source Code','Description']

In [69]:
data_1 = pd.read_csv("/home/jian/Projects/Red Nose Day/Data/20171122_TD-9697.csv",encoding = "ISO-8859-1")
data_2 = pd.read_csv("/home/jian/Projects/Red Nose Day/Data/ContributionReport-15530264647.csv",encoding = "ISO-8859-1")
data_3 = pd.read_csv("/home/jian/Projects/Red Nose Day/Data/export44-1355453973.csv",encoding = "ISO-8859-1")
data_4 = pd.read_csv("/home/jian/Projects/Red Nose Day/Data/export44-4092541519.csv",encoding = "ISO-8859-1")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (11,14,21,24,26,27,34,39,47,54,56,57,62,76,77) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
data_1['VANID']=data_1['VanID'].astype(int)
del data_1['VanID']
del data_1['FirstName']
del data_1['LastName']
data_2['VANID']=data_2['VANID'].astype(int)
del data_2['Home Phone']
data_3['VANID']=data_3['VANID'].astype(int)
data_4['VANID']=data_4['VANID'].astype(int)

In [71]:
US_state_filter=pd.read_csv("/home/jian/Docs/states.csv",dtype=str)
US_state_filter=US_state_filter.append({'State':'NA', 'Abbreviation':'NA'}, ignore_index=True)

In [72]:
data=pd.merge(data_1,data_2,on="VANID",how="right")
data=pd.merge(data,data_3,on="VANID",how="left")
data=pd.merge(data,data_4,on="VANID",how="left")
data=data[data['ActivistCodeName']!='Hand in Hand 2017']
data=data[data['Designation']!='Hand in Hand']
data=data[data['Designation Name']!='Hand in Hand']
data['Acknowledgement State/Province']=np.where(pd.isnull(data['Acknowledgement State/Province']),"NA",data['Acknowledgement State/Province'])
data['Acknowledgement State/Province']=data['Acknowledgement State/Province'].apply(lambda x: str(x).upper())
data=data[data['Acknowledgement State/Province'].isin(US_state_filter['Abbreviation'])]

data['Acknowledgement State/Province']=data['Acknowledgement State/Province'].apply(lambda x: x.strip())
data['Acknowledgement City']=data['Acknowledgement City'].apply(lambda x: str(x).strip().lower().capitalize())

In [73]:
matrix_column=['VANID','Date Received','Remaining Amount','Amount','Source Code','Acknowledgement City','Acknowledgement State/Province','Acknowledgement Zip/Postal','Contact Type',
              'Occupation','Origin Source Code','ActivistCodeType','ActivistCodeName','ActivistCodeDescription','Scope','Designation Name']

In [74]:
data_sub=data[matrix_column]

In [75]:
matrix_column

['VANID',
 'Date Received',
 'Remaining Amount',
 'Amount',
 'Source Code',
 'Acknowledgement City',
 'Acknowledgement State/Province',
 'Acknowledgement Zip/Postal',
 'Contact Type',
 'Occupation',
 'Origin Source Code',
 'ActivistCodeType',
 'ActivistCodeName',
 'ActivistCodeDescription',
 'Scope',
 'Designation Name']

In [88]:
writer=pd.ExcelWriter("/home/jian/Projects/Red Nose Day/Output from CRM/Sorted View.xlsx",engine='xlsxwriter')
def count_unique(x):
    return len(x.unique())

In [89]:
f = {'VANID':[count_unique], 'Amount':['sum']}

Df_Source_Code=data_sub[['VANID','Date Received','Amount','Source Code']]
Df_Source_Code['Source Code'].fillna("NA",inplace=True)
Df_Source_Code['Source Code']=np.where(Df_Source_Code['Source Code']=='NFG','Network for Good',Df_Source_Code['Source Code'])
# Remove "RND17 NoTV"
Df_Source_Code=Df_Source_Code[Df_Source_Code['Source Code']!='RND17 NoTV']
Df_Source_Code.drop_duplicates(inplace=True)
Df_Source_Code_Summary=Df_Source_Code.groupby('Source Code').agg(f)
Df_Source_Code_Summary.reset_index(inplace=True)
Df_Source_Code_Summary.columns=Df_Source_Code_Summary.columns.get_level_values(0)
Df_Source_Code_Summary=Df_Source_Code_Summary.sort_values(by=['VANID'],ascending=False)
Df_Source_Code_Summary['ID Pctg']=Df_Source_Code_Summary['VANID'].apply(lambda x: x/sum(Df_Source_Code_Summary['VANID']))
Df_Source_Code_Summary['Donation Pctg']=Df_Source_Code_Summary['Amount'].apply(lambda x: x/sum(Df_Source_Code_Summary['Amount']))
Df_Source_Code_Summary=pd.merge(Df_Source_Code_Summary,source_list_describe,on="Source Code",how='left')
Df_Source_Code_Summary.to_excel(writer,"Source_Code",index=False)
len(Df_Source_Code['VANID'].unique())


# Df_Source_Code_Summary_2=Df_Source_Code_Summary.sort_values(by=['Amount'],ascending=False)
# Df_Source_Code_Summary_2.to_excel(writer,"Source_Code_2")

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


250741

In [90]:
f = {'VANID':[count_unique], 'Amount':['sum']}

Media=data_sub[['VANID','Date Received','Amount','Source Code']]
Media['Source Code'].fillna("NA",inplace=True)
Media['Source Code']=np.where(Media['Source Code']=='NFG','Network for Good',Media['Source Code'])
# Remove "RND17 NoTV"
Media=Media[Media['Source Code']!='RND17 NoTV']
Media.drop_duplicates(inplace=True)

Media['Media']=np.where(Media['Source Code']=='PATRIOT',"Night of TV",
                                 np.where(Media['Source Code'].isin(['Network for Good','KIMBIA','RND17 Default Donation Form','Red Nose Day 2017']),"RND Website",
                                          np.where(Media['Source Code']=='mGive',"Text Message",
                                                   np.where(Media['Source Code'].isin(['CrowdRise','CROWDRISE_FR']),"CrowdRise Web",
                                                            np.where(Media['Source Code'].isin(['Lockbox']),"Check",
                                                                      np.where(Media['Source Code'].isin(['PPGF']),"PayPal's Web",
                                                                               np.where(Media['Source Code'].isin(['RND17 Post-Email Capture']),"Email",'Not Defined'
                                )))))))
del Media['Source Code']
Media_Summary=Media.groupby('Media').agg(f)
Media_Summary.reset_index(inplace=True)
Media_Summary.columns=Media_Summary.columns.get_level_values(0)
Media_Summary=Media_Summary.sort_values(by=['VANID'],ascending=False)
Media_Summary['ID Pctg']=Media_Summary['VANID'].apply(lambda x: x/sum(Media_Summary['VANID']))
Media_Summary['Donation Pctg']=Media_Summary['Amount'].apply(lambda x: x/sum(Media_Summary['Amount']))
Media_Summary.to_excel(writer,"Media",index=False)
# Media_Summary_2=Media_Summary.sort_values(by=['Amount'],ascending=False)
# Media_Summary_2.to_excel(writer,"Media_2")

len(Media['VANID'].unique())

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


250741

In [91]:
'Hand in Hand: Event & .com' in data['Origin Source Code'].unique()

False

In [92]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_Origin_Source_Code=data_sub[['VANID','Date Received','Amount','Origin Source Code']]
Df_Origin_Source_Code['Origin Source Code'].fillna("NA",inplace=True)
#
Df_Origin_Source_Code['Origin Source Code']=np.where(Df_Origin_Source_Code['Origin Source Code']=='NFG','Network for Good',Df_Origin_Source_Code['Origin Source Code'])
Df_Origin_Source_Code=Df_Origin_Source_Code[~Df_Origin_Source_Code['Origin Source Code'].isin(['RND17 NoTV','Hand in Hand Event','Hand in Hand: Event & .com'])]
Df_Origin_Source_Code.drop_duplicates(inplace=True)
Df_Origin_Source_Code_Summary=Df_Origin_Source_Code.groupby('Origin Source Code').agg(f)
Df_Origin_Source_Code_Summary.reset_index(inplace=True)
Df_Origin_Source_Code_Summary.columns=Df_Origin_Source_Code_Summary.columns.get_level_values(0)
Df_Origin_Source_Code_Summary=Df_Origin_Source_Code_Summary.sort_values(by=['VANID'],ascending=False)
Df_Origin_Source_Code_Summary['ID Pctg']=Df_Origin_Source_Code_Summary['VANID'].apply(lambda x: x/sum(Df_Origin_Source_Code_Summary['VANID']))
Df_Origin_Source_Code_Summary['Donation Pctg']=Df_Origin_Source_Code_Summary['Amount'].apply(lambda x: x/sum(Df_Origin_Source_Code_Summary['Amount']))
Df_Origin_Source_Code_Summary=pd.merge(Df_Origin_Source_Code_Summary,origin_source_list_describe,on="Origin Source Code",how='left')
Df_Origin_Source_Code_Summary.to_excel(writer,"Origin_Source_Code",index=False)

# Df_Origin_Source_Code_Summary_2=Df_Origin_Source_Code_Summary.sort_values(by=['Amount'],ascending=False)
# Df_Origin_Source_Code_Summary_2.to_excel(writer,"Origin_Source_Code_2")
len(Df_Origin_Source_Code['VANID'].unique())

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


292114

In [93]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_State=data_sub[['VANID','Date Received','Amount','Acknowledgement State/Province']]
Df_State.drop_duplicates(inplace=True)

Df_State_Summary=Df_State.groupby('Acknowledgement State/Province').agg(f)
Df_State_Summary.reset_index(inplace=True)
Df_State_Summary.columns=Df_State_Summary.columns.get_level_values(0)
Df_State_Summary=Df_State_Summary.sort_values(by=['VANID'],ascending=False)
Df_State_Summary['ID Pctg']=Df_State_Summary['VANID'].apply(lambda x: x/sum(Df_State_Summary['VANID']))
Df_State_Summary['Donation Pctg']=Df_State_Summary['Amount'].apply(lambda x: x/sum(Df_State_Summary['Amount']))
Df_State_Summary.to_excel(writer,"State",index=False)
# Df_State_Summary_2=Df_State_Summary.sort_values(by=['Amount'],ascending=False)
# Df_State_Summary_2.to_excel(writer,"State_2")
len(Df_State['VANID'].unique())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


307438

In [94]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_City=data_sub[['VANID','Date Received','Amount','Acknowledgement State/Province','Acknowledgement City']]

Df_City['Acknowledgement State/Province'].fillna("NA",inplace=True)
Df_City['Acknowledgement City'].fillna("NA",inplace=True)

Df_City.drop_duplicates(inplace=True)
Df_City_Summary=Df_City.groupby(['Acknowledgement State/Province','Acknowledgement City']).agg(f)
Df_City_Summary.reset_index(inplace=True)

Df_City_Summary.columns=Df_City_Summary.columns.get_level_values(0)

Df_City_Summary=Df_City_Summary.sort_values(by=['VANID'],ascending=False)
Df_City_Summary['ID Pctg']=Df_City_Summary['VANID'].apply(lambda x: x/sum(Df_City_Summary['VANID']))
Df_City_Summary['Donation Pctg']=Df_City_Summary['Amount'].apply(lambda x: x/sum(Df_City_Summary['Amount']))
Df_City_Summary.to_excel(writer,"City",index=False)
# Df_City_Summary_2=Df_City_Summary.sort_values(by=['Amount'],ascending=False)
# Df_City_Summary_2.to_excel(writer,"City_2")
len(Df_City['VANID'].unique())

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


307438

In [95]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_Zip=data_sub[['VANID','Date Received','Amount','Acknowledgement Zip/Postal','Acknowledgement State/Province']]
Df_Zip['Acknowledgement Zip/Postal']=Df_Zip['Acknowledgement Zip/Postal'].astype(str)
Df_Zip['Acknowledgement Zip/Postal']=Df_Zip['Acknowledgement Zip/Postal'].apply(lambda x: x.split('-')[0])
Df_Zip['Acknowledgement Zip/Postal']=np.where(Df_Zip['Acknowledgement Zip/Postal']=='nan','99999',Df_Zip['Acknowledgement Zip/Postal'])

Df_Zip['Acknowledgement Zip/Postal'].fillna("99999",inplace=True)
Df_Zip['Acknowledgement Zip/Postal']=Df_Zip['Acknowledgement Zip/Postal'].apply(lambda x: x.split('.')[0])
Df_Zip['Zip_len']=Df_Zip['Acknowledgement Zip/Postal'].apply(lambda x: len(x))
Df_Zip=Df_Zip[Df_Zip['Zip_len']<=5]

Df_Zip['Acknowledgement Zip/Postal']=Df_Zip['Acknowledgement Zip/Postal'].astype(int)
Df_Zip['Acknowledgement Zip/Postal']=Df_Zip['Acknowledgement Zip/Postal'].astype(str)
Df_Zip['Acknowledgement Zip/Postal']=np.where(len(Df_Zip['Acknowledgement Zip/Postal'])==1,"0000"+Df_Zip['Acknowledgement Zip/Postal'],
np.where(len(Df_Zip['Acknowledgement Zip/Postal'])==2,"000"+Df_Zip['Acknowledgement Zip/Postal'],
                                                np.where(len(Df_Zip['Acknowledgement Zip/Postal'])==3,"00"+Df_Zip['Acknowledgement Zip/Postal'],
                                                                                             np.where(len(Df_Zip['Acknowledgement Zip/Postal'])==4,"0"+Df_Zip['Acknowledgement Zip/Postal'],Df_Zip['Acknowledgement Zip/Postal']
                                                                                                     ))))

Df_Zip['Acknowledgement Zip/Postal']=np.where(Df_Zip['Acknowledgement Zip/Postal']=='99999','NA',Df_Zip['Acknowledgement Zip/Postal'])

Df_Zip.drop_duplicates(inplace=True)
Df_Zip_Summary=Df_Zip.groupby('Acknowledgement Zip/Postal').agg(f)
Df_Zip_Summary.reset_index(inplace=True)
Df_Zip_Summary.columns=Df_Zip_Summary.columns.get_level_values(0)
Df_Zip_Summary=Df_Zip_Summary.sort_values(by=['VANID'],ascending=False)
Df_Zip_Summary['ID Pctg']=Df_Zip_Summary['VANID'].apply(lambda x: x/sum(Df_Zip_Summary['VANID']))
Df_Zip_Summary['Donation Pctg']=Df_Zip_Summary['Amount'].apply(lambda x: x/sum(Df_Zip_Summary['Amount']))
Df_Zip_Summary.to_excel(writer,"Zip",index=False)

# Df_Zip_Summary_2=Df_Zip_Summary.sort_values(by=['Amount'],ascending=False)
# Df_Zip_Summary_2.to_excel(writer,"Zip_2")
len(Df_Zip['VANID'].unique())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

307437

In [96]:
len(Df_Zip['Acknowledgement Zip/Postal'].unique())

21071

In [98]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_Activist=data_sub[['VANID','Date Received','Amount','ActivistCodeType','ActivistCodeName']]
Df_Activist['ActivistCodeName'].fillna("NA",inplace=True)
Df_Activist.drop_duplicates(inplace=True)
Df_Activist_Summary=Df_Activist.groupby(['ActivistCodeType','ActivistCodeName']).agg(f)
Df_Activist_Summary.reset_index(inplace=True)
Df_Activist_Summary.columns=Df_Activist_Summary.columns.get_level_values(0)
Df_Activist_Summary=Df_Activist_Summary.sort_values(by=['VANID'],ascending=False)
Df_Activist_Summary['ID Pctg']=Df_Activist_Summary['VANID'].apply(lambda x: x/sum(Df_Activist_Summary['VANID']))
Df_Activist_Summary['Donation Pctg']=Df_Activist_Summary['Amount'].apply(lambda x: x/sum(Df_Activist_Summary['Amount']))
Df_Activist_Summary=pd.merge(Df_Activist_Summary,source_list,on='ActivistCodeName',how='left')


Df_Activist_Summary.to_excel(writer,"Activist_Name",index=False)
# Df_Activist_Summary_2=Df_Activist_Summary.sort_values(by=['Amount'],ascending=False)
# Df_Activist_Summary_2.to_excel(writer,"Activist_Name_2")
len(Df_Activist['VANID'].unique())

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


307438

In [99]:
f = {'VANID':[count_unique], 'Amount':['sum']}
Df_Activist_Code=data_sub[['VANID','Date Received','Amount','ActivistCodeType']]
Df_Activist_Code['ActivistCodeType'].fillna("NA",inplace=True)
Df_Activist_Code.drop_duplicates(inplace=True)
Df_Activist_Code_Summary=Df_Activist_Code.groupby('ActivistCodeType').agg(f)
Df_Activist_Code_Summary.reset_index(inplace=True)
Df_Activist_Code_Summary.columns=Df_Activist_Code_Summary.columns.get_level_values(0)
Df_Activist_Code_Summary=Df_Activist_Code_Summary.sort_values(by=['VANID'],ascending=False)
Df_Activist_Code_Summary['ID Pctg']=Df_Activist_Code_Summary['VANID'].apply(lambda x: x/sum(Df_Activist_Code_Summary['VANID']))
Df_Activist_Code_Summary['Donation Pctg']=Df_Activist_Code_Summary['Amount'].apply(lambda x: x/sum(Df_Activist_Code_Summary['Amount']))
Df_Activist_Code_Summary.to_excel(writer,"Activist_Type",index=False)
# Df_Activist_Code_Summary_2=Df_Activist_Code_Summary.sort_values(by=['Amount'],ascending=False)
# Df_Activist_Code_Summary_2.to_excel(writer,"Activist_Type_2")
len(Df_Activist_Code['VANID'].unique())

/usr/local/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


307438

In [100]:
writer.save()